# 데이터 클리닝

In [1]:
import warnings
warnings.filterwarnings("ignore", category=SyntaxWarning)


In [2]:
import pandas as pd
df = pd.read_csv('titanic.csv')

In [3]:
# Name 컬럼에서 Title을 추출해서 Title 컬럼에 저장
df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

# Title의 값이 대중적이지 않은 것은 Other 값으로 치환
title_counts = df['Title'].value_counts()
rare_titles = title_counts[title_counts < 10].index
df['Title'] = df['Title'].replace(rare_titles, 'Other')

In [4]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,Other
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,Miss
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,Miss
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,Mr


In [5]:
# 승객번호, 이름, 티켓, cabin 컬럼을 삭제해
df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)



In [6]:
# 성별을 0, 1로 바꿔줘
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})

In [7]:
# age 컬럼의 결측치를 title, 생존유무, 객실등급, 성별을 참고하여 중앙값으로 채워줘

# 각 title, 생존유무, 객실등급, 성별 조합에 대한 age의 중앙값을 계산
median_ages = df.groupby(['Title', 'Survived', 'Pclass', 'Sex'])['Age'].median()

# 결측치를 채우는 함수 정의
def fill_age(row):
    if pd.isnull(row['Age']):
        return median_ages[row['Title'], row['Survived'], row['Pclass'], row['Sex']]
    else:
        return row['Age']

# age 컬럼의 결측치를 채워줘
df['Age'] = df.apply(fill_age, axis=1)


In [8]:

# Title, Embarked 컬럼에 원핫인코딩 적용해
df = pd.get_dummies(df, columns=['Title', 'Embarked'])
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Other,Embarked_C,Embarked_Q,Embarked_S
0,0,3,0,22.0,1,0,7.2500,False,False,True,False,False,False,False,True
1,1,1,1,38.0,1,0,71.2833,False,False,False,True,False,True,False,False
2,1,3,1,26.0,0,0,7.9250,False,True,False,False,False,False,False,True
3,1,1,1,35.0,1,0,53.1000,False,False,False,True,False,False,False,True
4,0,3,0,35.0,0,0,8.0500,False,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,0,27.0,0,0,13.0000,False,False,False,False,True,False,False,True
887,1,1,1,19.0,0,0,30.0000,False,True,False,False,False,False,False,True
888,0,3,1,18.0,1,2,23.4500,False,True,False,False,False,False,False,True
889,1,1,0,26.0,0,0,30.0000,False,False,True,False,False,True,False,False


In [9]:
df.to_csv('titanic_cleaned.csv', index=False)

In [10]:
df = pd.read_csv('titanic_cleaned.csv')

# 나이 예측 모델 만들어보기

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [13]:
# 나이 예측을 위한 입력 변수와 타겟 변수 설정
X = df.drop(columns=['Age'])
y = df['Age']

In [14]:

# 학습 데이터와 테스트 데이터로 분리 (시드 42로 설정)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# 선형 회귀 모델 생성 및 학습
model = LinearRegression()
model

LinearRegression()

In [16]:
model.fit(X_train, y_train)

LinearRegression()

In [17]:
# 테스트 데이터로 예측
y_pred = model.predict(X_test)

# 예측 성능 평가 (평균 절대 오차, R2 스코어)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

results = pd.DataFrame([
    ['base_line', mse, mae, r2]
], columns=['test_name', 'Mean Squared Error', 'Mean Absolute Error', 'R2 Score'])

results

,test_name,Mean Squared Error,Mean Absolute Error,R2 Score
0,base_line,107.64497,7.776262,0.451054


# 스케일링 적용하여 모델 테스트

In [18]:
from sklearn.preprocessing import MinMaxScaler

# Min-Max 스케일링 적용
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# 학습 데이터와 테스트 데이터로 분리 (시드 42로 설정)
X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 선형 회귀 모델 생성 및 학습
model_scaled = LinearRegression()
model_scaled.fit(X_train_scaled, y_train)

# 테스트 데이터로 예측
y_pred_scaled = model_scaled.predict(X_test_scaled)

# 예측 성능 평가 (평균 절대 오차, R2 스코어)
mae_scaled = mean_absolute_error(y_test, y_pred_scaled)
r2_scaled = r2_score(y_test, y_pred_scaled)
mse_scaled = mean_squared_error(y_test, y_pred_scaled)

# 결과 기록
results_scaled = pd.DataFrame([
    ['min_max_scaled', mse_scaled, mae_scaled, r2_scaled]
], columns=['test_name', 'Mean Squared Error', 'Mean Absolute Error', 'R2 Score'])

# 기존 결과와 합치기
results = pd.concat([results, results_scaled], ignore_index=True)
results






,test_name,Mean Squared Error,Mean Absolute Error,R2 Score
0,base_line,107.64497,7.776262,0.451054
1,min_max_scaled,107.64497,7.776262,0.451054


# 기능들을 함수화 하여 여러 모델 시험해보기

In [19]:
def evaluate_model(model, X_train, X_test, y_train, y_test):
    # 모델 학습
    model.fit(X_train, y_train)

    # 테스트 데이터로 예측
    y_pred = model.predict(X_test)

    # 예측 성능 평가 (평균 절대 오차, R2 스코어, 평균 제곱 오차)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    return mse, mae, r2

In [20]:
def add_evaluation_results(results, test_name, mse, mae, r2):
    new_results = pd.DataFrame([
        [test_name, mse, mae, r2]
    ], columns=['test_name', 'Mean Squared Error', 'Mean Absolute Error', 'R2 Score'])

    return pd.concat([results, new_results], ignore_index=True)


In [21]:
from sklearn.preprocessing import StandardScaler

# 데이터 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 모델 평가
mse_scaled, mae_scaled, r2_scaled = evaluate_model(model, X_train_scaled, X_test_scaled, y_train, y_test)

# 결과 기록 및 기존 결과와 합치기
results = add_evaluation_results(results, 'standard_scaled', mse_scaled, mae_scaled, r2_scaled)
results



,test_name,Mean Squared Error,Mean Absolute Error,R2 Score
0,base_line,107.64497,7.776262,0.451054
1,min_max_scaled,107.64497,7.776262,0.451054
2,standard_scaled,107.64497,7.776262,0.451054


- 데이터 스케일링 통해서는 더 이상 성능향상을 할 수 없음을 확인
- 그렇다면 여러 모델을 시험해보자

In [23]:
import xgboost as xgb
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor

# 모델 리스트와 이름 정의
models = [
    (xgb.XGBRegressor(), 'xgboost'),
    (Ridge(), 'ridge'),
    (Lasso(), 'lasso'),
    (RandomForestRegressor(), 'random_forest')
]

# 각 모델에 대해 평가 및 결과 기록
for model, name in models:
    mse, mae, r2 = evaluate_model(model, X_train, X_test, y_train, y_test)
    results = add_evaluation_results(results, name, mse, mae, r2)

In [24]:
# mae, r2score 기준으로 내림차순 정렬
results = results.sort_values(by=['Mean Absolute Error', 'R2 Score']).reset_index(drop=True)
results


,test_name,Mean Squared Error,Mean Absolute Error,R2 Score
0,xgboost,108.681423,7.323744,0.445769
1,random_forest,107.993673,7.334195,0.449276
2,ridge,104.872435,7.696827,0.465193
3,min_max_scaled,107.644970,7.776262,0.451054
4,standard_scaled,107.644970,7.776262,0.451054
5,base_line,107.644970,7.776262,0.451054
6,lasso,139.426364,9.068622,0.288982
